<a href="https://colab.research.google.com/github/nimosaidhaji/WeightsandBiases/blob/main/Weights_and_biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install wandb scikit-learn pandas



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import wandb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report




In [ ]:
# Initialize wandb
wandb.login()



<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Initialize a new wandb project
wandb.init(project="phishing-url-detection")



wandb: Currently logged in as: saidnimo103 (saidnimo103-none). Use `wandb login --relogin` to force relogin


In [16]:

# Load the dataset from Google Drive or any other source
# Make sure to upload your urlset.csv to the Colab environment
from google.colab import files
uploaded = files.upload()

# Function to read the CSV in chunks and handle errors
def read_csv_in_chunks(file_path):
    chunks = []
    chunk_size = 1000
    try:
        for chunk in pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size):
            chunks.append(chunk)
    except UnicodeDecodeError:
        for chunk in pd.read_csv(file_path, encoding='latin1', chunksize=chunk_size):
            chunks.append(chunk)
    except Exception as e:
        print(f"Error reading chunk: {e}")
    return pd.concat(chunks, ignore_index=True)

# Load the dataset in chunks
df = read_csv_in_chunks('urlset.csv')

# Print the columns of the DataFrame to inspect the data
print("Columns in the DataFrame:", df.columns)

# Ensure the necessary columns are present
if 'domain' not in df.columns or 'label' not in df.columns:
    raise KeyError("The dataset must contain 'url' and 'label' columns.")

# Inspect the label distribution
print("Label distribution:\n", df['label'].value_counts())

# Check if the dataset contains at least two classes
if len(df['label'].value_counts()) < 2:
    # Add a new class to the dataset
    new_data = {
        'domain': ['http://example-nonphishing.com', 'http://safe-example.com'],
        'label': [0, 0]  # Assuming 0 is the label for non-phishing
    }
    df_new_class = pd.DataFrame(new_data)
    df = pd.concat([df, df_new_class], ignore_index=True)

# Print the updated label distribution
print("Updated label distribution:\n", df['label'].value_counts())

# If the dataset is imbalanced, consider sampling
# For example, downsample the majority class or upsample the minority class
# Here we show an example of downsampling the majority class
min_class_count = df['label'].value_counts().min()
df_balanced = df.groupby('label').apply(lambda x: x.sample(min_class_count)).reset_index(drop=True)


Saving urlset.csv to urlset.csv
Error reading chunk: Error tokenizing data. C error: Expected 14 fields in line 18259, saw 15

Columns in the DataFrame: Index(['domain', 'ranking', 'mld_res', 'mld.ps_res', 'card_rem', 'ratio_Rrem',
       'ratio_Arem', 'jaccard_RR', 'jaccard_RA', 'jaccard_AR', 'jaccard_AA',
       'jaccard_ARrd', 'jaccard_ARrem', 'label'],
      dtype='object')
Label distribution:
 label
1.0    18000
Name: count, dtype: int64
Updated label distribution:
 label
1.0    18000
0.0        2
Name: count, dtype: int64


In [17]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['domain'])
y = df['label']


In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [19]:
# Train logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)


LogisticRegression()

In [20]:

# Predict and evaluate logistic regression model
y_pred_logreg = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
wandb.log({"logreg_accuracy": accuracy_logreg, "logreg_classification_report": classification_report(y_test, y_pred_logreg, output_dict=True)})



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# Train decision tree model
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)



DecisionTreeClassifier()

In [22]:
# Predict and evaluate decision tree model
y_pred_tree = decision_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
wandb.log({"tree_accuracy": accuracy_tree, "tree_classification_report": classification_report(y_test, y_pred_tree, output_dict=True)})



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# Finish wandb run
wandb.finish()

print("Logistic Regression Accuracy: ", accuracy_logreg)
print("Decision Tree Accuracy: ", accuracy_tree)

logreg_accuracy,▁
tree_accuracy,▁
logreg_accuracy,0.99972
tree_accuracy,0.99972


Logistic Regression Accuracy:  0.9997222993612885
Decision Tree Accuracy:  0.9997222993612885
